In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
from collections import Counter
import numpy as np
import sys
import matplotlib.pyplot as plt

# LOAD DATA

In [2]:
# df_usr_view_pay = pd.read_csv('dataset/user_view_pay_df_with_more_features.csv', index_col=[0, 1])
df_usr_view_pay = pd.read_csv('dataset/df_view_pay_with_month_week_aqi_weather.csv', index_col=[0, 1])
df_usr_view_pay

view_cnt  pay_cnt  aqi  weather  wind   tmp  weekday_0  \
shop_id time_stamp                                                           
1       2015-10-10         0      188   64      0.0   0.0  17.0          0   
        2015-10-11         0      272   68      6.0   0.0  16.5          0   
        2015-10-12         0      257   85      1.0   0.0  17.5          1   
        2015-10-13         0      239   84      5.0   0.0  17.5          0   
        2015-10-14         0      277  143      0.0   0.0  18.0          0   
        2015-10-15         0      227  164      6.0   0.0  19.0          0   
        2015-10-16         0      242  135      1.0   0.0  20.5          0   
        2015-10-17         0      293   78      5.0   0.0  20.0          0   
        2015-10-18         0      291   66      5.0   0.0  20.0          0   
        2015-10-19         0      256   67      0.0   0.0  19.5          1   
        2015-10-20         0      262   76      7.0   0.0  19.5          0   
        2015-10-21         0      271   69      0.0   0.0  20.5          0   
        2015-10-22         0      260   85      0.0   0.0  20.5          0   
        2015-10-23         0      240  102      7.0   0.0  20.5          0   
        2015-10-24         0      291   77      3.0   0.0  21.0          0   
        2015-10-25         0      286   71      7.0   0.0  21.0          0   
        2015-10-26         0      264   83     27.0   0.0  20.0          1   
        2015-10-27         0      295   66      8.0   0.0  16.5          0   
        2015-10-28         0      310   86     16.0   0.0  16.0          0   
        2015-10-29         0      212   76      4.0   0.0  15.0          0   
        2015-10-30         0      331   51      3.0   0.0  14.5          0   
        2015-10-31         0      347   40     16.0   0.0  14.5          0   
        2015-11-01         0      381   66      8.0   0.0  10.5          0   
        2015-11-02         0      332   80      5.0   0.0  12.5          1   
        2015-11-03         0      404  111      5.0   0.0  16.0          0   
        2015-11-04         0      392  112      3.0   0.0  16.5          0   
        2015-11-05         0      496   89      8.0   0.0  19.0          0   
        2015-11-06         0      430   63     16.0   0.0  22.0          0   
        2015-11-07         0      351   46      4.0   0.0  17.5          0   
        2015-11-08         0      288   42      4.0   0.0  11.5          0   
...                      ...      ...  ...      ...   ...   ...        ...   
2000    2016-10-02         4       39   65      0.0   2.0  26.0          0   
        2016-10-03         3       50   37      4.0   5.0  24.0          1   
        2016-10-04         8       29   46      0.0   1.0  23.5          0   
        2016-10-05         3       35   56      0.0   1.0  24.0          0   
        2016-10-06         4       76   54      4.0   5.0  24.5          0   
        2016-10-07        11       68   43      4.0   2.0  21.5          0   
        2016-10-08        16      158   36      2.0   5.0  19.5          0   
        2016-10-09        24      176   42      0.0   5.0  19.0          0   
        2016-10-10        35      161   42      3.0   2.0  18.0          1   
        2016-10-11        24      163   46      3.0   2.0  20.0          0   
        2016-10-12        33      159   52      0.0   2.0  21.5          0   
        2016-10-13        35      177   67      0.0   2.0  20.5          0   
        2016-10-14        14      148   58      2.0   2.0  20.5          0   
        2016-10-15         9       78   55      2.0   1.0  21.5          0   
        2016-10-16         4       60   60      2.0   2.0  23.0          0   
        2016-10-17        16      163   80      2.0   2.0  22.0          1   
        2016-10-18        18      157   61      2.0   1.0  22.0          0   
        2016-10-19        14      164   50      2.0   1.0  22.0          0   
        2016-10-20        35      165   40

In [3]:
df_usr_view_pay.columns

Index(['view_cnt', 'pay_cnt', 'aqi', 'weather', 'wind', 'tmp', 'weekday_0',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12'],
      dtype='object')

In [4]:
quant_features = ['view_cnt', 'pay_cnt', 'aqi', 'weather', 'wind', 'tmp']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = df_usr_view_pay[each].mean(), df_usr_view_pay[each].std()
    scaled_features[each] = [mean, std]
    df_usr_view_pay.loc[:, each] = (df_usr_view_pay[each] - mean)/std
scaled_features

{'aqi': [69.10689907646486, 41.82863241974223],
 'pay_cnt': [108.25559585194213, 130.7345540633734],
 'tmp': [20.06643747988851, 9.000948791616628],
 'view_cnt': [15.710891994111511, 42.47333460898241],
 'weather': [12.999961137218264, 29.293252471358795],
 'wind': [1.3992824375980313, 2.446529545336942]}

In [5]:
# df_usr_view_pay = df_usr_view_pay.drop(['time_stamp', 'view_cnt'], axis=1)
df_usr_view_pay

view_cnt   pay_cnt       aqi   weather      wind  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972 -0.122091 -0.443787 -0.571946   
        2015-10-11 -0.369900  1.252495 -0.026463 -0.238962 -0.571946   
        2015-10-12 -0.369900  1.137759  0.379957 -0.409649 -0.571946   
        2015-10-13 -0.369900  1.000075  0.356050 -0.273099 -0.571946   
        2015-10-14 -0.369900  1.290741  1.766567 -0.443787 -0.571946   
        2015-10-15 -0.369900  0.908286  2.268616 -0.238962 -0.571946   
        2015-10-16 -0.369900  1.023023  1.575311 -0.409649 -0.571946   
        2015-10-17 -0.369900  1.413126  0.212608 -0.273099 -0.571946   
        2015-10-18 -0.369900  1.397828 -0.074277 -0.273099 -0.571946   
        2015-10-19 -0.369900  1.130110 -0.050370 -0.443787 -0.571946   
        2015-10-20 -0.369900  1.176004  0.164794 -0.204824 -0.571946   
        2015-10-21 -0.369900  1.244846 -0.002556 -0.443787 -0.571946   
        2015-10-22 -0.369900  1.160706  0.379957 -0.443787 -0.571946   
        2015-10-23 -0.369900  1.007724  0.786378 -0.204824 -0.571946   
        2015-10-24 -0.369900  1.397828  0.188701 -0.341374 -0.571946   
        2015-10-25 -0.369900  1.359582  0.045258 -0.204824 -0.571946   
        2015-10-26 -0.369900  1.191303  0.332143  0.477927 -0.571946   
        2015-10-27 -0.369900  1.428424 -0.074277 -0.170686 -0.571946   
        2015-10-28 -0.369900  1.543161  0.403865  0.102414 -0.571946   
        2015-10-29 -0.369900  0.793550  0.164794 -0.307237 -0.571946   
        2015-10-30 -0.369900  1.703791 -0.432883 -0.341374 -0.571946   
        2015-10-31 -0.369900  1.826177 -0.695861  0.102414 -0.571946   
        2015-11-01 -0.369900  2.086246 -0.074277 -0.170686 -0.571946   
        2015-11-02 -0.369900  1.711440  0.260422 -0.273099 -0.571946   
        2015-11-03 -0.369900  2.262175  1.001541 -0.273099 -0.571946   
        2015-11-04 -0.369900  2.170386  1.025448 -0.341374 -0.571946   
        2015-11-05 -0.369900  2.965891  0.475586 -0.170686 -0.571946   
        2015-11-06 -0.369900  2.461051 -0.145998  0.102414 -0.571946   
        2015-11-07 -0.369900  1.856773 -0.552418 -0.307237 -0.571946   
        2015-11-08 -0.369900  1.374881 -0.648047 -0.307237 -0.571946   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742 -0.098184 -0.443787  0.245539   
        2016-10-03 -0.299268 -0.445602 -0.767582 -0.307237  1.471765   
        2016-10-04 -0.181547 -0.606233 -0.552418 -0.443787 -0.163204   
        2016-10-05 -0.299268 -0.560338 -0.313348 -0.443787 -0.163204   
        2016-10-06 -0.275723 -0.246726 -0.361162 -0.307237  1.471765   
        2016-10-07 -0.110914 -0.307919 -0.624139 -0.307237  0.245539   
        2016-10-08  0.006807  0.380499 -0.791489 -0.375512  1.471765   
        2016-10-09  0.195160  0.518183 -0.648047 -0.443787  1.471765   
        2016-10-10  0.454146  0.403447 -0.648047 -0.341374  0.245539   
        2016-10-11  0.195160  0.418745 -0.552418 -0.341374  0.245539   
        2016-10-12  0.407058  0.388148 -0.408976 -0.443787  0.245539   
        2016-10-13  0.454146  0.525832 -0.050370 -0.443787  0.245539   
        2016-10-14 -0.040282  0.304008 -0.265533 -0.375512  0.245539   
        2016-10-15 -0.158002 -0.231428 -0.337255 -0.375512 -0.163204   
        2016-10-16 -0.275723 -0.369111 -0.217719 -0.375512  0.245539   
        2016-10-17  0.006807  0.418745  0.260422 -0.375512  0.245539   
        2016-10-18  0.053895  0.372850 -0.193812 -0.375512 -0.163204   
        2016-10-19 -0.040282  0.426394 -0.456790 -0.375512 -0.163204   
        2016-10-20  0.454146  0.434043 -0.695861 -0.307237  0.245539   
        2016-10-21 -0.063826  0.227518 -0.863210 -0.375512  0.245539   
        2016-10-22 -0.228635 -0.323217 -1.150095  0.853440  3.924219   
        2016-10-23 -0.275723 -0.376760 -0.743675 -0.307237  0.245539   
        2016-10-24  0.077439  0.411096 -0.193812 -0.341374 -

In [6]:
df_shop_info = pd.read_csv('dataset/normed_shop_info_features.csv', index_col=[0])
df_shop_info

,city_idx,location_id,per_pay,score,comment_cnt,shop_level_0,shop_level_1,shop_level_2,cate_1_idx_0,cate_1_idx_1,...,cate_3_idx_34,cate_3_idx_35,cate_3_idx_36,cate_3_idx_37,cate_3_idx_38,cate_3_idx_39,cate_3_idx_40,cate_3_idx_41,cate_3_idx_42,cate_3_idx_43
1,1.192024,0.899196,-0.430197,1.397435e+00,2.742265e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1.402179,-1.545979,1.478704,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.456482,0.568606,-0.950806,3.411693e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.561560,-0.604840,1.305167,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,-0.594291,-0.953299,-1.471415,-7.150959e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0.771715,1.655681,0.437485,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-0.016366,-1.304737,0.437485,3.411693e-01,1.814729e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,-0.594291,-1.039670,0.958095,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,-0.594291,1.599093,0.784558,1.397435e+00,-9.678796e-01,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10,1.717411,-1.444717,-0.083124,-1.501043e-14,4.805603e-15,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# SPLIT DATA TO TWO SETS

In [7]:
validation_ratio = 0.1
num_samples = 2000

In [8]:
validation_idx = np.random.randint(1, num_samples+1, size=int(validation_ratio*num_samples))
print(validation_idx.shape)
validation_idx

(200,)


array([1338, 1663,  922, 1779, 1969,  200,  565, 1814, 1080,  573,  986,
       1307,  913, 1925,  255,  435, 1164,  508, 1884, 1483, 1972,   44,
        588, 1015,  494, 1592,  544, 1019,  927,  272,  383,  436,  940,
       1737, 1672, 1531, 1142, 1698,  563,  430,  671,  816, 1831, 1318,
        271,  489, 1888, 1174, 1974,  107,  847, 1041, 1002,  622,  837,
        725, 1005,  333, 1737,  316, 1475,  450,  589, 1359, 1215,  443,
        446, 1645,  419,  208,  379,  701,  581, 1283, 1381, 1756, 1689,
        703,  476,   19, 1693, 1050, 1823,  223, 1147,  829, 1150, 1224,
        960,  449, 1887, 1166,  370,  331,  765,  136, 1514,  982, 1149,
        132, 1201, 1098, 1586, 1779,  823, 1897, 1367, 1779, 1618, 1544,
        638,  600, 1117, 1194, 1767, 1559, 1839, 1374, 1883, 1459, 1575,
        253,  412,  868, 1733, 1999,  687,  354,  311, 1545,  706,  156,
        269,  422,  858, 1213,   41,  793,  687,  743, 1891, 1604, 1474,
        454,  288, 1831, 1139, 1549, 1463, 1306,  2

In [9]:
len(set(validation_idx))

187

In [10]:
df_usr_train = df_usr_view_pay.drop(list(validation_idx), axis=0)
df_usr_train.reset_index(inplace=True)
df_usr_train.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_train

view_cnt   pay_cnt       aqi   weather      wind  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972 -0.122091 -0.443787 -0.571946   
        2015-10-11 -0.369900  1.252495 -0.026463 -0.238962 -0.571946   
        2015-10-12 -0.369900  1.137759  0.379957 -0.409649 -0.571946   
        2015-10-13 -0.369900  1.000075  0.356050 -0.273099 -0.571946   
        2015-10-14 -0.369900  1.290741  1.766567 -0.443787 -0.571946   
        2015-10-15 -0.369900  0.908286  2.268616 -0.238962 -0.571946   
        2015-10-16 -0.369900  1.023023  1.575311 -0.409649 -0.571946   
        2015-10-17 -0.369900  1.413126  0.212608 -0.273099 -0.571946   
        2015-10-18 -0.369900  1.397828 -0.074277 -0.273099 -0.571946   
        2015-10-19 -0.369900  1.130110 -0.050370 -0.443787 -0.571946   
        2015-10-20 -0.369900  1.176004  0.164794 -0.204824 -0.571946   
        2015-10-21 -0.369900  1.244846 -0.002556 -0.443787 -0.571946   
        2015-10-22 -0.369900  1.160706  0.379957 -0.443787 -0.571946   
        2015-10-23 -0.369900  1.007724  0.786378 -0.204824 -0.571946   
        2015-10-24 -0.369900  1.397828  0.188701 -0.341374 -0.571946   
        2015-10-25 -0.369900  1.359582  0.045258 -0.204824 -0.571946   
        2015-10-26 -0.369900  1.191303  0.332143  0.477927 -0.571946   
        2015-10-27 -0.369900  1.428424 -0.074277 -0.170686 -0.571946   
        2015-10-28 -0.369900  1.543161  0.403865  0.102414 -0.571946   
        2015-10-29 -0.369900  0.793550  0.164794 -0.307237 -0.571946   
        2015-10-30 -0.369900  1.703791 -0.432883 -0.341374 -0.571946   
        2015-10-31 -0.369900  1.826177 -0.695861  0.102414 -0.571946   
        2015-11-01 -0.369900  2.086246 -0.074277 -0.170686 -0.571946   
        2015-11-02 -0.369900  1.711440  0.260422 -0.273099 -0.571946   
        2015-11-03 -0.369900  2.262175  1.001541 -0.273099 -0.571946   
        2015-11-04 -0.369900  2.170386  1.025448 -0.341374 -0.571946   
        2015-11-05 -0.369900  2.965891  0.475586 -0.170686 -0.571946   
        2015-11-06 -0.369900  2.461051 -0.145998  0.102414 -0.571946   
        2015-11-07 -0.369900  1.856773 -0.552418 -0.307237 -0.571946   
        2015-11-08 -0.369900  1.374881 -0.648047 -0.307237 -0.571946   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742 -0.098184 -0.443787  0.245539   
        2016-10-03 -0.299268 -0.445602 -0.767582 -0.307237  1.471765   
        2016-10-04 -0.181547 -0.606233 -0.552418 -0.443787 -0.163204   
        2016-10-05 -0.299268 -0.560338 -0.313348 -0.443787 -0.163204   
        2016-10-06 -0.275723 -0.246726 -0.361162 -0.307237  1.471765   
        2016-10-07 -0.110914 -0.307919 -0.624139 -0.307237  0.245539   
        2016-10-08  0.006807  0.380499 -0.791489 -0.375512  1.471765   
        2016-10-09  0.195160  0.518183 -0.648047 -0.443787  1.471765   
        2016-10-10  0.454146  0.403447 -0.648047 -0.341374  0.245539   
        2016-10-11  0.195160  0.418745 -0.552418 -0.341374  0.245539   
        2016-10-12  0.407058  0.388148 -0.408976 -0.443787  0.245539   
        2016-10-13  0.454146  0.525832 -0.050370 -0.443787  0.245539   
        2016-10-14 -0.040282  0.304008 -0.265533 -0.375512  0.245539   
        2016-10-15 -0.158002 -0.231428 -0.337255 -0.375512 -0.163204   
        2016-10-16 -0.275723 -0.369111 -0.217719 -0.375512  0.245539   
        2016-10-17  0.006807  0.418745  0.260422 -0.375512  0.245539   
        2016-10-18  0.053895  0.372850 -0.193812 -0.375512 -0.163204   
        2016-10-19 -0.040282  0.426394 -0.456790 -0.375512 -0.163204   
        2016-10-20  0.454146  0.434043 -0.695861 -0.307237  0.245539   
        2016-10-21 -0.063826  0.227518 -0.863210 -0.375512  0.245539   
        2016-10-22 -0.228635 -0.323217 -1.150095  0.853440  3.924219   
        2016-10-23 -0.275723 -0.376760 -0.743675 -0.307237  0.245539   
        2016-10-24  0.077439  0.411096 -0.193812 -0.341374 -

In [11]:
df_usr_train.index.levels[0]

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 2000],
           dtype='int64', name='shop_id', length=1813)

In [12]:
df_usr_validation = df_usr_view_pay.loc[list(validation_idx)]
df_usr_validation.reset_index(inplace=True)
df_usr_validation.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_validation

view_cnt   pay_cnt       aqi   weather      wind  \
shop_id time_stamp                                                     
19      2015-10-10 -0.369900 -0.675075  0.499493 -0.238962 -0.571946   
        2015-10-11 -0.369900 -0.369111  0.212608 -0.409649 -0.571946   
        2015-10-12 -0.369900 -0.437953  0.977634 -0.409649 -0.571946   
        2015-10-13 -0.369900 -0.575637  1.407961 -0.409649 -0.571946   
        2015-10-14 -0.369900 -0.598584  1.694846 -0.238962 -0.571946   
        2015-10-15 -0.369900 -0.361462 -0.408976 -0.409649 -0.571946   
        2015-10-16 -0.369900 -0.376760 -0.241626 -0.409649 -0.571946   
        2015-10-17 -0.369900 -0.460900 -0.385069 -0.273099 -0.571946   
        2015-10-18 -0.369900 -0.346164 -0.217719 -0.443787 -0.571946   
        2015-10-19 -0.369900 -0.193182 -0.217719 -0.443787 -0.571946   
        2015-10-20 -0.369900 -0.330866 -0.313348 -0.443787 -0.571946   
        2015-10-21 -0.369900 -0.277322 -0.145998 -0.273099 -0.571946   
        2015-10-22 -0.369900 -0.208480  1.001541 -0.273099 -0.571946   
        2015-10-23 -0.369900 -0.208480 -0.145998 -0.443787 -0.571946   
        2015-10-24 -0.369900  0.464639 -0.098184 -0.443787 -0.571946   
        2015-10-25 -0.369900  0.089834 -0.098184 -0.443787 -0.571946   
        2015-10-26 -0.369900 -0.040201 -0.050370  0.102414 -0.571946   
        2015-10-27 -0.369900 -0.147288  0.714656 -0.238962 -0.571946   
        2015-10-28 -0.369900 -0.200831  0.571214 -0.034136 -0.571946   
        2015-10-29 -0.369900 -0.284971 -0.313348 -0.375512 -0.571946   
        2015-10-30 -0.369900 -0.131990 -0.504604 -0.341374 -0.571946   
        2015-10-31 -0.369900  0.028641 -0.743675 -0.068274 -0.571946   
        2015-11-01 -0.369900  0.984777 -0.456790  0.068276 -0.571946   
        2015-11-02 -0.369900  0.418745  2.005638 -0.273099 -0.571946   
        2015-11-03 -0.369900  0.824146  1.384054 -0.443787 -0.571946   
        2015-11-04 -0.369900  0.556428 -0.408976 -0.375512 -0.571946   
        2015-11-05 -0.369900  1.229548 -0.289440 -0.034136 -0.571946   
        2015-11-06 -0.369900  1.306039 -0.719768 -0.375512 -0.571946   
        2015-11-07 -0.369900  0.870041 -0.122091  0.614477 -0.571946   
        2015-11-08 -0.369900  1.023023  1.192798 -0.375512 -0.571946   
...                      ...       ...       ...       ...       ...   
1999    2016-10-02  0.124528  2.063298  0.619028 -0.409649 -0.163204   
        2016-10-03  0.148072  1.780282  0.738563 -0.443787 -0.163204   
        2016-10-04 -0.016737  1.764984  0.929820 -0.375512  0.245539   
        2016-10-05  0.053895  1.520213 -0.456790 -0.341374 -0.163204   
        2016-10-06  0.100984  1.956211 -0.050370 -0.375512 -0.163204   
        2016-10-07  0.053895  2.208631  0.140887 -0.375512 -0.163204   
        2016-10-08  0.265793  1.703791 -0.576325 -0.375512 -0.163204   
        2016-10-09  0.195160  1.504915 -0.911024 -0.375512 -0.163204   
        2016-10-10 -0.040282  1.757335 -0.887117 -0.375512 -0.163204   
        2016-10-11  0.265793  1.244846 -0.695861 -0.375512 -0.163204   
        2016-10-12  0.171616  1.543161 -0.504604 -0.375512 -0.163204   
        2016-10-13  0.148072  1.543161 -0.217719 -0.341374 -0.163204   
        2016-10-14  0.218704  1.887369 -0.074277 -0.375512 -0.163204   
        2016-10-15  0.407058  2.988838  0.116980 -0.341374 -0.163204   
        2016-10-16  0.265793  3.287152  1.144984 -0.375512 -0.163204   
        2016-10-17  0.124528  1.535511  0.379957 -0.443787 -0.163204   
        2016-10-18  0.006807  1.351933  0.523400 -0.443787 -0.163204   
        2016-10-19  0.312881  1.298390  0.714656 -0.341374 -0.163204   
        2016-10-20  0.148072  1.321337  0.882006 -0.341374 -0.163204   
        2016-10-21  0.171616  1.780282  0.953727 -0.375512 -0.163204   
        2016-10-22  0.383514  2.575787  1.049355 -0.375512 -0.163204   
        2016-10-23  0.100984  2.522244 -1.030560 -0.375512 -0.163204   
        2016-10-24 -0.299268  1.466670 -0.743675 -0.341374 -

In [13]:
len(df_usr_validation.index.levels[0])

187

# START NN!

### Define Data Generator

In [14]:
predict_days = 14

In [68]:
class Config(object):
    num_steps = 60
    batch_size = 50
    num_lstm_hidden = 32
    num_lstm_layers = 2
    num_lstm_features = 25
    num_shop_features = 75
#     keep_prob = 0.75
config = Config()

In [69]:
df_usr = df_usr_train


In [70]:
len(list(df_usr.index.levels[0]))

1813

In [71]:
a = [np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]])]
print(a)
b = np.stack(a)
print(b)
b.shape

[array([[1, 2],
       [3, 4]]), array([[5, 6],
       [7, 8]])]
[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]


(2, 2, 2)

In [72]:
ss = df_usr_view_pay.loc[1]
ss.reset_index(drop=True)

,view_cnt,pay_cnt,aqi,weather,wind,tmp,weekday_0,weekday_1,weekday_2,weekday_3,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,-0.369900,0.609972,-0.122091,-0.443787,-0.571946,-0.340679,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-0.369900,1.252495,-0.026463,-0.238962,-0.571946,-0.396229,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-0.369900,1.137759,0.379957,-0.409649,-0.571946,-0.285130,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-0.369900,1.000075,0.356050,-0.273099,-0.571946,-0.285130,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-0.369900,1.290741,1.766567,-0.443787,-0.571946,-0.229580,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,-0.369900,0.908286,2.268616,-0.238962,-0.571946,-0.118481,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,-0.369900,1.023023,1.575311,-0.409649,-0.571946,0.048169,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,-0.369900,1.413126,0.212608,-0.273099,-0.571946,-0.007381,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,-0.369900,1.397828,-0.074277,-0.273099,-0.571946,-0.007381,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,-0.369900,1.130110,-0.050370,-0.443787,-0.571946,-0.062931,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [73]:
def data_generator(config, df_usr, df_shop, random=True):
    batch_size = config.batch_size
    num_steps = config.num_steps
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        if random:
            np.random.shuffle(ID_list)
            batch_ID_list = ID_list[0:batch_size]
        else:
            batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        _shop_info = []
        for ID in batch_ID_list:
            each_shop_info = df_shop.loc[ID].values
            _shop_info.append(each_shop_info)
            
            each_shop_usr = df_usr.loc[ID]
            each_shop_usr = each_shop_usr.reset_index(drop=True)
            _m = each_shop_usr.shape[0] - predict_days
            assert num_steps < _m, 'lower num_steps!'
            _s_idx = np.random.randint(0, _m-num_steps, 1)[0]
            _e_idx = _s_idx + num_steps
#             print('_m: {}, _s_idx: {}, _e_idx: {}'.format(_m, _s_idx, _e_idx))
            _features.append(each_shop_usr.loc[_s_idx:_e_idx-1].values)
            _labels.append(each_shop_usr.loc[_e_idx:(_e_idx+predict_days-1), 'pay_cnt'].values)
        
#         for label in _labels:
#             print(label.shape)
        
        features = np.stack(_features)
        labels = np.stack(_labels)
        shop_info = np.stack(_shop_info)
#         print('features.shape: {}, labels.shape: {}, shop_info.shape: {}'.format(features.shape, labels.shape, shop_info.shape))
        yield features, labels, shop_info, batch_ID_list


In [74]:
f, l, s, _ = data_generator(config, df_usr_train, df_shop_info).__next__()
l.shape

(50, 14)

### Define LSTM

In [75]:
import tensorflow as tf

In [76]:
tf.__version__

'1.0.0'

In [77]:
def nn_lstm(x_tensor, config, _keep_prob):
    num_steps = config.num_steps
    num_lstm_hidden = config.num_lstm_hidden
    num_lstm_layers = config.num_lstm_layers
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x_tensor = tf.transpose(x_tensor, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x_tensor = tf.reshape(x_tensor, [-1, 1])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x_tensor = tf.split(x_tensor, num_or_size_splits=num_steps, axis=0)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.LSTMCell(num_lstm_hidden)
    lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, input_keep_prob=_keep_prob, output_keep_prob=_keep_prob)
    lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell]*num_lstm_layers)
    

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x_tensor, dtype=tf.float32, sequence_length=None)
#     print('outputs.shape: {}, {}, states.shape: {}'.format(len(outputs),outputs[-1].shape, len(states)))
    return outputs[-1]

In [78]:
def merge(lstm_out, shop_info, config):
    num_lstm_features = config.num_lstm_features
    num_lstm_hidden = config.num_lstm_hidden
    lstm_out = tf.reshape(lstm_out, shape=[-1, num_lstm_features*num_lstm_hidden])
    return tf.concat([lstm_out, shop_info], axis=1)

In [79]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    w_fc = tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1], num_outputs], stddev=0.1))
    b_fc = tf.Variable(tf.zeros(num_outputs))
    fc = tf.add(tf.matmul(x_tensor, w_fc), b_fc)
    fc = tf.nn.relu(fc)
    return fc

In [80]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    w_o = tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1], num_outputs], stddev=0.5))
    b_o = tf.Variable(tf.zeros(num_outputs))
    output = tf.add(tf.matmul(x_tensor, w_o), b_o)
    return output

In [81]:
def whole_nn(x_seq, x_shop, config, _keep_prob):

    lstm_out = nn_lstm(x_seq, config, _keep_prob)
    
    merge_feature = merge(lstm_out, x_shop, config)
    
    print('merge_feature.shape: {}'.format(merge_feature.shape))
    fc_1 = fully_conn(merge_feature, 1024)
    fc_1 = tf.nn.dropout(fc_1, _keep_prob)
    
    fc_2 = fully_conn(fc_1, 1024)
#     fc_2 = tf.nn.dropout(fc_2, _keep_prob)
    
    fc_3 = fully_conn(fc_2, 1024)
    fc_2 = tf.nn.dropout(fc_2, _keep_prob)
    
    regression_out = output(fc_3, predict_days)
    
    return regression_out

In [82]:
def reg_loss(y_pred, y_g):
    _tmp = tf.abs(tf.div(tf.subtract(y_pred, y_g), tf.add(y_pred, y_g)))
    return tf.reduce_mean(_tmp, name='T_loss')
#     m, n = y_pred.shape
#     _sub = tf.subtract(y_pred, y_g)
#     _add = tf.add(y_pred, y_g)
#     _sum = 0
#     for i in xrange(m):
#         for j in xrange(n):
#             if _add[i, j] == 0:
#                 continue
#             else:
#                 _sum += abs(_sub[i, j] / _add[i, j])
#     return _sum / (m*n)

In [83]:
##############################
## Build the Neural Network ##
##############################

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x_seq = tf.placeholder(tf.float32, shape=[None, config.num_steps, config.num_lstm_features], name='x_seq')
x_shop = tf.placeholder(tf.float32, shape=[None, config.num_shop_features], name='x_shop')
y = tf.placeholder(tf.float32, shape=[None, predict_days], name='y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Model
reg_out = whole_nn(x_seq, x_shop, config, keep_prob)

# Name reg_out Tensor, so that is can be loaded from disk after training
reg_out = tf.identity(reg_out, name='reg_out')

# Loss and Optimizer
cost = tf.reduce_mean(tf.pow(tf.subtract(reg_out, y), 2), name='cost')
T_loss = reg_loss(reg_out, y)
optimizer = tf.train.AdamOptimizer().minimize(cost)

# # Accuracy
# correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


merge_feature.shape: (?, 875)


In [84]:
epochs = 100
keep_probablity = 0.75

In [85]:
save_model_path = 'model/customer_prediction'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for batch_id, (features, labels, shop_info, _) in enumerate(data_generator(config, df_usr_view_pay, df_shop_info)):
            train_feed_dict = {x_seq: features, x_shop: shop_info, y: labels, keep_prob: keep_probablity}
            sess.run(optimizer, feed_dict=train_feed_dict)
            train_cost = sess.run(cost, feed_dict=train_feed_dict)
            train_T_loss = sess.run(T_loss, feed_dict=train_feed_dict)
            print('epoch: {:>2}, batch: {:>2}, train_cost: {:>8.4f}, train_T_loss: {:>8.4f}'.format(
                epoch+1, batch_id+1, train_cost, train_T_loss))
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)


epoch:  1, batch:  1, train_cost: 161.0879, train_T_loss:   1.0938
epoch:  1, batch:  2, train_cost: 113.2760, train_T_loss:   1.2056
epoch:  1, batch:  3, train_cost:  39.8172, train_T_loss:   1.3230
epoch:  1, batch:  4, train_cost:  42.0570, train_T_loss:   4.9037
epoch:  1, batch:  5, train_cost:  62.3545, train_T_loss:   2.5064
epoch:  1, batch:  6, train_cost:  57.1956, train_T_loss:   1.7046
epoch:  1, batch:  7, train_cost:  51.0334, train_T_loss:   8.9935
epoch:  1, batch:  8, train_cost:  30.3596, train_T_loss:   3.0266
epoch:  1, batch:  9, train_cost:  20.4328, train_T_loss:   1.4020
epoch:  1, batch: 10, train_cost:  18.8645, train_T_loss:   2.3623
epoch:  1, batch: 11, train_cost:  16.3470, train_T_loss:   1.8310
epoch:  1, batch: 12, train_cost:  22.0001, train_T_loss:   2.4657
epoch:  1, batch: 13, train_cost:  17.9937, train_T_loss:   2.1131
epoch:  1, batch: 14, train_cost:  21.4744, train_T_loss:   1.4086
epoch:  1, batch: 15, train_cost:  20.0074, train_T_loss:   1.

# CHECKPOINT

In [86]:
def final_data_generator(config, df_usr, df_shop, random=True):
    batch_size = config.batch_size
    num_steps = config.num_steps
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        if random:
            np.random.shuffle(ID_list)
            batch_ID_list = ID_list[0:batch_size]
        else:
            batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        _shop_info = []
        for ID in batch_ID_list:
            each_shop_info = df_shop.loc[ID].values
            _shop_info.append(each_shop_info)
            
            each_shop_usr = df_usr.loc[ID]
            each_shop_usr = each_shop_usr.reset_index(drop=True)
            _e_idx = each_shop_usr.shape[0]
            _s_idx = _e_idx - num_steps
#             print('_m: {}, _s_idx: {}, _e_idx: {}'.format(_m, _s_idx, _e_idx))
            _features.append(each_shop_usr.loc[_s_idx:_e_idx-1].values)
        
        features = np.stack(_features)
        shop_info = np.stack(_shop_info)
#         print('features.shape: {}, labels.shape: {}, shop_info.shape: {}'.format(features.shape, labels.shape, shop_info.shape))
        yield features, shop_info, batch_ID_list

In [87]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf

save_model_path = 'model/customer_prediction'

test_config = Config()
test_config.batch_size = 50

"""
Test the saved model against the test dataset
"""
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(save_model_path + '.meta')
    loader.restore(sess, save_model_path)

    # Get Tensors from loaded model
    loaded_x_seq = loaded_graph.get_tensor_by_name('x_seq:0')
    loaded_x_shop = loaded_graph.get_tensor_by_name('x_shop:0')
    loaded_y = loaded_graph.get_tensor_by_name('y:0')
    loaded_reg_out = loaded_graph.get_tensor_by_name('reg_out:0')
    loaded_cost = loaded_graph.get_tensor_by_name('cost:0')
    loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    # Get accuracy in batches for memory limitations
    test_batch_cost_total = 0
    test_batch_count = 0
    result_list = []
    reg_out_list = []
    for features, labels, shop_info, batch_ID_list in data_generator(test_config, df_usr_view_pay, df_shop_info, random=False):
        test_feed_dict={loaded_x_seq: features, loaded_x_shop: shop_info, loaded_y: labels, loaded_keep_prob: 1.}
        test_batch_cost_total += sess.run(loaded_cost, feed_dict=test_feed_dict)
        test_batch_count += 1
        test_reg_out = sess.run(loaded_reg_out, feed_dict=test_feed_dict)
        tmp_list = [pd.DataFrame(np.array([test_reg_out[i,:], labels[i,:]]).T, columns=['pred', 'label']) for i in range(test_config.batch_size)]
        result_list.append(pd.concat(tmp_list, keys=batch_ID_list, names=['shop_id', 'predict_day']))
        
        reg_out_list.append(pd.DataFrame(test_reg_out))
        
    df_result = pd.concat(result_list)
    
    final_reg_out_list = []
    for features, shop_info, batch_ID_list in final_data_generator(test_config, df_usr_view_pay, df_shop_info, random=False):
        test_feed_dict={loaded_x_seq: features, loaded_x_shop: shop_info, loaded_keep_prob: 1.}
        final_reg_out = sess.run(loaded_reg_out, feed_dict=test_feed_dict)
        final_reg_out_list.append(pd.DataFrame(final_reg_out))
   
    df_pred = pd.concat(final_reg_out_list, ignore_index=True)
    

    print('Testing Cost: {}\n'.format(test_batch_cost_total/test_batch_count))


Testing Cost: 0.23097468614578248



In [88]:
def negtive_check(df, op=True):
    a = unnormed_df_pred < 0
    if a.sum().sum() > 0:
        print('Find negative!')
        if op:
            df[a] = 0
    else:
        print('no negative number~')

In [89]:
df_result

pred     label
shop_id predict_day                    
1       0            0.766043  0.663516
        1            0.815579  0.831795
        2            0.803487  0.915935
        3            0.617673  0.533481
        4            0.682893  0.709410
        5            0.629626  0.816497
        6            0.791591  0.854743
        7            0.758231  0.648217
        8            0.801210  0.640568
        9            0.648025  0.732357
        10           0.752208  0.824146
        11           0.691391  0.908286
        12           0.823709  0.808848
        13           0.697486  0.609972
2       0           -0.093788 -0.506795
        1           -0.033166 -0.369111
        2           -0.128814  0.181623
        3           -0.100599  0.426394
        4           -0.160540 -0.277322
        5           -0.115486 -0.315568
        6           -0.128154 -0.292620
        7           -0.153544 -0.254375
        8           -0.049029 -0.361462
        9           -0.111569  0.273412
        10          -0.114815  0.258114
        11          -0.132735 -0.193182
        12          -0.071305 -0.453251
        13          -0.130564 -0.284971
3       0           -0.124210 -0.193182
        1           -0.137497 -0.361462
...                       ...       ...
1998    12           0.909606  2.223929
        13           0.879380  0.548779
1999    0            1.805385  0.961830
        1            1.969570  1.328986
        2            1.888765  0.961830
        3            1.693516  1.214250
        4            1.782547  1.604353
        5            1.712441  1.153057
        6            1.758364  1.336635
        7            2.039928  1.283092
        8            1.880625  0.587025
        9            1.751595  1.191303
        10           1.772963  1.390179
        11           1.613544  1.091864
        12           1.739307  1.818528
        13           1.871387  1.023023
2000    0            0.248678  0.388148
        1            0.261232  0.434043
        2            0.228985  0.242816
        3            0.275891 -0.476198
        4            0.190342 -0.284971
        5            0.228436  0.441692
        6            0.185933  0.587025
        7            0.269991  0.434043
        8            0.202202  0.357552
        9            0.225054  0.395797
        10           0.276071 -0.131990
        11           0.230186  0.112781
        12           0.273285  1.015374
        13           0.261883  0.732357

[28000 rows x 2 columns]

In [90]:
scaled_features

{'aqi': [69.10689907646486, 41.82863241974223],
 'pay_cnt': [108.25559585194213, 130.7345540633734],
 'tmp': [20.06643747988851, 9.000948791616628],
 'view_cnt': [15.710891994111511, 42.47333460898241],
 'weather': [12.999961137218264, 29.293252471358795],
 'wind': [1.3992824375980313, 2.446529545336942]}

In [91]:
mean, std = scaled_features['pay_cnt']
unnormed_df_result = df_result *std + mean
unnormed_df_result

pred  label
shop_id predict_day                   
1       0            208.403949  195.0
        1            214.879991  217.0
        2            213.299119  228.0
        3            189.006743  178.0
        4            197.533297  201.0
        5            190.569514  215.0
        6            211.743883  220.0
        7            207.382554  193.0
        8            213.001473  192.0
        9            192.974883  204.0
        10           206.595235  216.0
        11           198.644258  227.0
        12           215.942787  214.0
        13           199.441123  188.0
2       0             95.994304   42.0
        1            103.919634   60.0
        2             91.415122  132.0
        3             95.103816  164.0
        4             87.267488   72.0
        5             93.157543   67.0
        6             91.501485   70.0
        7             88.182081   75.0
        8            101.845825   61.0
        9             93.669689  144.0
        10            93.245277  142.0
        11            90.902547   83.0
        12            98.933593   49.0
        13            91.186307   71.0
3       0             92.017062   83.0
        1             90.280031   61.0
...                         ...    ...
1998    12           227.172466  399.0
        13           223.220978  180.0
1999    0            344.281751  234.0
        1            365.746410  282.0
        2            355.182396  234.0
        3            329.656657  267.0
        4            341.296129  318.0
        5            332.130818  259.0
        6            338.134524  283.0
        7            374.944699  276.0
        8            354.118221  185.0
        9            337.249605  264.0
        10           340.043067  290.0
        11           319.201549  251.0
        12           335.643158  346.0
        13           352.910588  242.0
2000    0            140.766446  159.0
        1            142.407706  165.0
        2            138.191900  140.0
        3            144.324021   46.0
        4            133.139893   71.0
        5            138.120138  166.0
        6            132.563512  185.0
        7            143.552727  165.0
        8            134.690414  155.0
        9            137.677930  160.0
        10           144.347593   91.0
        11           138.348913  123.0
        12           143.983373  241.0
        13           142.492806  204.0

[28000 rows x 2 columns]

In [92]:
_pred = unnormed_df_result.pred.values
_label = unnormed_df_result.label.values
loss = np.mean(np.abs((_pred-_label)/(_pred+_label)))
loss

0.19964164931279474

### OUTPUT PREDICT

In [93]:
mean, std = scaled_features['pay_cnt']
unnormed_df_pred = df_pred *std + mean
unnormed_df_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,227.757843,249.183807,238.993408,215.492523,216.420685,221.844101,236.058823,255.611786,228.080902,216.482666,242.080597,221.653564,233.261948,241.699799
1,99.145729,104.336853,90.774948,96.886864,89.301506,95.391296,92.285233,91.257767,102.696617,93.425636,93.093719,91.456299,99.572952,90.888893
2,98.186287,95.261070,92.696381,99.273811,81.499168,93.083298,95.573608,85.636360,94.868301,92.070412,93.378891,94.315086,94.390053,99.225853
3,139.657364,141.830948,146.056473,153.508835,139.458008,143.750580,134.428238,150.695312,141.043015,145.114929,154.437698,141.185150,147.604752,149.539124
4,273.990967,275.014313,267.985229,254.168243,269.628815,271.427917,267.883453,288.784302,254.307129,277.716949,260.454193,273.660187,278.604828,278.056274
5,96.855957,97.941566,92.381462,98.165947,83.564468,90.762993,96.638306,84.342613,102.912155,93.522964,97.062881,96.185066,99.752319,97.095535
6,158.923859,154.391434,156.911926,161.510757,149.778061,155.446365,150.758408,153.386139,151.718414,150.409958,159.581146,161.245850,164.822311,157.092850
7,77.325684,74.276917,73.155487,81.061638,67.498985,76.528976,77.674690,72.009048,74.613792,75.390137,80.332214,77.253410,74.874771,82.164200
8,310.671417,316.792755,307.332611,283.704132,303.596375,295.787415,305.952789,332.542999,306.410492,305.352264,296.976440,289.260620,306.558228,312.334259
9,115.794487,111.564163,111.724197,128.770325,102.713638,123.412857,112.379051,112.989868,112.813118,115.519775,119.650650,117.734451,116.367851,126.633347


In [94]:
negtive_check(unnormed_df_pred)
unnormed_df_pred


no negative number~


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,227.757843,249.183807,238.993408,215.492523,216.420685,221.844101,236.058823,255.611786,228.080902,216.482666,242.080597,221.653564,233.261948,241.699799
1,99.145729,104.336853,90.774948,96.886864,89.301506,95.391296,92.285233,91.257767,102.696617,93.425636,93.093719,91.456299,99.572952,90.888893
2,98.186287,95.261070,92.696381,99.273811,81.499168,93.083298,95.573608,85.636360,94.868301,92.070412,93.378891,94.315086,94.390053,99.225853
3,139.657364,141.830948,146.056473,153.508835,139.458008,143.750580,134.428238,150.695312,141.043015,145.114929,154.437698,141.185150,147.604752,149.539124
4,273.990967,275.014313,267.985229,254.168243,269.628815,271.427917,267.883453,288.784302,254.307129,277.716949,260.454193,273.660187,278.604828,278.056274
5,96.855957,97.941566,92.381462,98.165947,83.564468,90.762993,96.638306,84.342613,102.912155,93.522964,97.062881,96.185066,99.752319,97.095535
6,158.923859,154.391434,156.911926,161.510757,149.778061,155.446365,150.758408,153.386139,151.718414,150.409958,159.581146,161.245850,164.822311,157.092850
7,77.325684,74.276917,73.155487,81.061638,67.498985,76.528976,77.674690,72.009048,74.613792,75.390137,80.332214,77.253410,74.874771,82.164200
8,310.671417,316.792755,307.332611,283.704132,303.596375,295.787415,305.952789,332.542999,306.410492,305.352264,296.976440,289.260620,306.558228,312.334259
9,115.794487,111.564163,111.724197,128.770325,102.713638,123.412857,112.379051,112.989868,112.813118,115.519775,119.650650,117.734451,116.367851,126.633347


In [95]:

int_df_pred = unnormed_df_pred.apply(np.round).astype(np.int16)
int_df_pred

int_df_pred.insert(value=range(1, 2001), loc=0, column='id')
int_df_pred

int_df_pred.to_csv('pred/pred_1.csv', header=False, index=False)
int_df_pred

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,228,249,239,215,216,222,236,256,228,216,242,222,233,242
1,2,99,104,91,97,89,95,92,91,103,93,93,91,100,91
2,3,98,95,93,99,81,93,96,86,95,92,93,94,94,99
3,4,140,142,146,154,139,144,134,151,141,145,154,141,148,150
4,5,274,275,268,254,270,271,268,289,254,278,260,274,279,278
5,6,97,98,92,98,84,91,97,84,103,94,97,96,100,97
6,7,159,154,157,162,150,155,151,153,152,150,160,161,165,157
7,8,77,74,73,81,67,77,78,72,75,75,80,77,75,82
8,9,311,317,307,284,304,296,306,333,306,305,297,289,307,312
9,10,116,112,112,129,103,123,112,113,113,116,120,118,116,127
